# Decision Tree Regressor

```plaintext

INITIALIZE MinMaxScaler for scaler_X and scaler_y

SET X to 'YearMonthWeek' values from agg_df, reshaped to (-1, 1)
SET y to 'Rate' values from agg_df, reshaped to (-1, 1)

FIT and TRANSFORM X and y with their respective scalers, store the results in X_scaled and y_scaled respectively

SET param_grid with range of 'max_depth', 'min_samples_split' and 'min_samples_leaf' values

CREATE DecisionTreeRegressor object dt_reg

CREATE TimeSeriesSplit object tscv with 5 splits

CREATE GridSearchCV object grid with dt_reg, param_grid and tscv, and fit it on X_scaled and y_scaled

PRINT best parameters and best RMSE of grid

RETRIEVE best model from grid

DEFINE FUNCTION compute_accuracy that takes a row:
    IF 'RATE_actual' in row equals to 0:
        RETURN NaN
    ELSE:
        CALCULATE error and error_proportion
        RETURN (1 - error_proportion) * 100
    ENDIF
END FUNCTION

SET weeks to 12

GET max 'YearMonthWeek' from agg_df and STORE it in last_date
GENERATE date range from last_date for weeks+1 periods with 'W' frequency, starting from the second date and STORE it in future_dates
CONVERT future_dates to ordinal and RESHAPE it to (-1, 1), STORE it in future_dates_ordinal

SCALE future_dates_ordinal using scaler_X and STORE it in future_dates_ordinal_scaled

PREDICT using best_model on future_dates_ordinal_scaled and STORE it in forecasted_values_scaled

INVERSE TRANSFORM forecasted_values_scaled using scaler_y and RESHAPE it to (-1, 1), STORE it in forecasted_values

CREATE DataFrame df_forecasted from future_dates and forecasted_values

ROUND 'Rate' in df_forecasted to 2 decimal places


# ARIMA

```plaintext
DEFINE FUNCTION calculate_RMSE that takes y_true, y_pred:
    RETURN the square root of mean_squared_error of y_true and y_pred
END FUNCTION

DEFINE FUNCTION ARIMA_Execute that takes data:
    # Fit an auto_arima model
    SET arima_model to auto_arima with data, trace=True, error_action='ignore', suppress_warnings=True, stepwise=True, alpha=0.07, m=7

    # Print the summary of the model
    PRINT the summary of arima_model

    # Forecast
    SET predictions to arima_model's prediction with n_periods equals to the length of data

    # Calculate the RMSE
    SET RMSE_ARIMA_data to calculate_RMSE with data and predictions
    PRINT "RMSE: %.3f" % RMSE_ARIMA_data

    RETURN arima_model and RMSE_ARIMA_data
END FUNCTION

# Get the 'Rate' column values
SET data to 'Rate' values from agg_df

# Run ARIMA model
SET model and ARIMA_rmse to ARIMA_Execute with data